# Formants are easy to measure; resonances, not so much: Lessons from Klatt (1986)
### D. H. Whalen, Wei-Rong Chen, Christine H. Shadle, Sean A. Fulop
#### August 2022 (JASA)

### One-liner
...

### Defining Terms
**formant** - 
local maxima in the wideband sound spectrum \
**vocal tract resonance** - 
... \
**wideband spectrogram** - 
... \
**narrowband spectrogram** - 
... \
**reassigned spectrogram** - 
... \

### Notes

#### Author Recommendations
- acknowledge limitations of current analyses regarding influence of F0 and limits of granularity
- ...